In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
import time
from tensorflow.keras.callbacks import History
from tensorflow.keras.callbacks import ReduceLROnPlateau, TensorBoard
from tensorflow.keras import metrics
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from keras.callbacks import TensorBoard
import pickle

In [2]:
sns.set()

In [3]:
tf.__version__

'2.15.0'

In [4]:
# Import the necessary libraries
import platform
import tensorflow as tf

# Print Python version
print("Python version: ", platform.python_version())

# Print the TensorFlow version
print("TensorFlow version: ", tf.__version__)

# Print the current working directory
import os
print("Current working directory: ", os.getcwd())


Python version:  3.11.7
TensorFlow version:  2.15.0
Current working directory:  /root/.jupyter/张彤/模型


In [5]:
# View Linux system information This command is used in Linux to obtain the system kernel and other information. Executing this command returns detailed information about the operating system, including the kernel version, system architecture, and so on.
!uname -a

Linux w3q2ulc9.vm 5.15.0-60-generic #66-Ubuntu SMP Fri Jan 20 14:29:49 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux


In [6]:
!nvidia-smi

Sat Oct 26 10:52:30 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090        On  | 00000000:09:00.0 Off |                  Off |
|  0%   27C    P8              10W / 450W |      3MiB / 24564MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
EPOCHS = 50
IMAGE_SIZE = (224,224)
IMAGE_PATH = "../data"
LEARNING_RATE = 1e-4
BATCH_SIZE = 64

In [8]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    IMAGE_PATH,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

Found 14080 files belonging to 100 classes.
Using 11264 files for training.


2024-10-26 10:52:33.203247: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-26 10:52:33.203595: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-26 10:52:33.320421: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [9]:
vaild_ds = tf.keras.utils.image_dataset_from_directory(
    IMAGE_PATH,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

Found 14080 files belonging to 100 classes.
Using 2816 files for validation.


In [10]:
class_names = train_ds.class_names
class_names

['1',
 '10',
 '100',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '2',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '3',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '4',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '5',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '6',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '7',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '8',
 '80',
 '81',
 '82',
 '83',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '9',
 '90',
 '91',
 '92',
 '93',
 '94',
 '95',
 '96',
 '97',
 '98',
 '99']

In [11]:
# Define the mean and standard deviation
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# Convert the mean and standard deviation to TensorFlow tensors
mean_tensor = tf.constant(mean, dtype=tf.float32)
std_tensor = tf.constant(std, dtype=tf.float32)

# Define a function to perform a standardized operation
def normalize_image(image):
    return (image - mean_tensor) / std_tensor


In [12]:
# Image enhancement definition
train_image_augment = tf.keras.Sequential(
    [
        tf.keras.layers.Rescaling(1 / 255.0), # normalization
        tf.keras.layers.RandomRotation(factor=0.2), # Random rotation
        tf.keras.layers.RandomFlip(), # Random flip
    ]
)

valid_image_augment = tf.keras.Sequential(
    [
        tf.keras.layers.Rescaling(1 / 255.0), # normalization
    ]
)

In [13]:
# Input processing method
def process_train_input(images, labels):
    return train_image_augment(images), labels

def process_valid_input(images, labels):
    return valid_image_augment(images), labels

In [14]:
# one-hot encoding and type conversion
def convert_types_and_encode(x, y):
    y = tf.cast(y, tf.int32)  # Convert the label type to an integer
    y_one_hot = tf.one_hot(y, 100)  # Apply one-hot encoding
    return x, y_one_hot

In [15]:
# Apply data set preprocessing
train_ds = train_ds.map(convert_types_and_encode) # Convert data types and encodings
train_ds = train_ds.map(process_train_input, num_parallel_calls=tf.data.AUTOTUNE) # Application enhancement
train_ds = train_ds.prefetch(tf.data.AUTOTUNE) # Optimized loading

vaild_ds = vaild_ds.map(convert_types_and_encode) # Convert data types and encodings
vaild_ds = vaild_ds.map(process_valid_input, num_parallel_calls=tf.data.AUTOTUNE) # Application enhancement
vaild_ds = vaild_ds.prefetch(tf.data.AUTOTUNE) # Optimized loading

In [16]:
base_model = tf.keras.applications.ResNet152V2(include_top=True,weights='imagenet',input_shape=(*IMAGE_SIZE,3))

In [17]:
for layer in base_model.layers:
  print(layer.name)

input_1
conv1_pad
conv1_conv
pool1_pad
pool1_pool
conv2_block1_preact_bn
conv2_block1_preact_relu
conv2_block1_1_conv
conv2_block1_1_bn
conv2_block1_1_relu
conv2_block1_2_pad
conv2_block1_2_conv
conv2_block1_2_bn
conv2_block1_2_relu
conv2_block1_0_conv
conv2_block1_3_conv
conv2_block1_out
conv2_block2_preact_bn
conv2_block2_preact_relu
conv2_block2_1_conv
conv2_block2_1_bn
conv2_block2_1_relu
conv2_block2_2_pad
conv2_block2_2_conv
conv2_block2_2_bn
conv2_block2_2_relu
conv2_block2_3_conv
conv2_block2_out
conv2_block3_preact_bn
conv2_block3_preact_relu
conv2_block3_1_conv
conv2_block3_1_bn
conv2_block3_1_relu
conv2_block3_2_pad
conv2_block3_2_conv
conv2_block3_2_bn
conv2_block3_2_relu
max_pooling2d
conv2_block3_3_conv
conv2_block3_out
conv3_block1_preact_bn
conv3_block1_preact_relu
conv3_block1_1_conv
conv3_block1_1_bn
conv3_block1_1_relu
conv3_block1_2_pad
conv3_block1_2_conv
conv3_block1_2_bn
conv3_block1_2_relu
conv3_block1_0_conv
conv3_block1_3_conv
conv3_block1_out
conv3_block2_pre

In [18]:
trainable = False
for layer in base_model.layers:
    if layer.name == 'conv5_block1_preact_bn':
        trainable = True
    layer.trainable = trainable

In [19]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100, activation='softmax'),
])

In [20]:
# tf.keras.utils.plot_model(model)

In [21]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet152v2 (Functional)    (None, 1000)              60380648  
                                                                 
 flatten (Flatten)           (None, 1000)              0         
                                                                 
 dense (Dense)               (None, 1024)              1025024   
                                                                 
 batch_normalization (Batch  (None, 1024)              4096      
 Normalization)                                                  
                                                                 
 dense_1 (Dense)             (None, 100)               102500    
                                                                 
Total params: 61512268 (234.65 MB)
Trainable params: 18149452 (69.23 MB)
Non-trainable params: 43362816 (165.42 MB)
____

In [22]:
# Definition optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)

# Defined loss function
loss_fn = tf.keras.losses.CategoricalCrossentropy()

# Define evaluation indicators
metrics = [
    tf.keras.metrics.CategoricalAccuracy(),
    tf.keras.metrics.Precision(),
    tf.keras.metrics.Recall(),
]

# Compilation model
model.compile(optimizer=optimizer, loss=loss_fn, metrics=metrics)


In [23]:
log_dir = "../Running result/resnet152v2/resnet152v2"

In [24]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [25]:
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.5, patience=2, min_lr=1e-8)

In [26]:
history = model.fit(train_ds,epochs=EPOCHS,validation_data=vaild_ds,callbacks=[lr_scheduler,tensorboard_callback])

Epoch 1/50


2024-10-26 10:52:56.569796: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-10-26 10:52:56.794111: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-10-26 10:52:59.325509: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f0fd58b7830 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-10-26 10:52:59.325558: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2024-10-26 10:52:59.325567: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2024-10-26 10:52:59.339210: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1729911179.523167    1622 device_comp

176/176 [==============================] - 55s 198ms/step - loss: 3.7854 - categorical_accuracy: 0.1940 - precision: 0.7558 - recall: 0.0346 - val_loss: 4.3789 - val_categorical_accuracy: 0.3111 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - lr: 1.0000e-04
Epoch 2/50
176/176 [==============================] - 31s 171ms/step - loss: 2.4306 - categorical_accuracy: 0.4132 - precision: 0.7646 - recall: 0.2154 - val_loss: 3.7569 - val_categorical_accuracy: 0.4496 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - lr: 1.0000e-04
Epoch 3/50
176/176 [==============================] - 29s 165ms/step - loss: 1.8000 - categorical_accuracy: 0.5338 - precision: 0.8037 - recall: 0.3578 - val_loss: 2.5283 - val_categorical_accuracy: 0.5312 - val_precision: 1.0000 - val_recall: 3.5511e-04 - lr: 1.0000e-04
Epoch 4/50
176/176 [==============================] - 30s 168ms/step - loss: 1.4189 - categorical_accuracy: 0.6247 - precision: 0.8257 - recall: 0.4760 - val_loss: 1.5025 - val_categorica

In [29]:
import pandas as pd  # Import the Pandas library
import os

# Assume the model has been trained and 'history' contains the training history

# Define save path
model_dir = "../Running result/resent152v2"
excel_file_path = os.path.join(model_dir, "resent152v2.xlsx")  # Path to save the Excel file
model_file_path = os.path.join(model_dir, "resnet152v2.h5")  # Path to save the model

# Check if the save directory exists, create it if not
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# Save the training history to an Excel file
history_df = pd.DataFrame(history.history)  
history_df.to_excel(excel_file_path, index=False)
print(f"Training history saved to: {excel_file_path}")

# Save the model to the specified path
model.save(model_file_path)
print(f"Model saved to: {model_file_path}")


Training history saved to: ../Running result/resent152v2/resent152v2.xlsx
Model saved to: ../Running result/resent152v2/resnet152v2.h5


In [ ]:
import os
import time
import numpy as np
import tensorflow as tf
import random
from PIL import Image
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Input

# Force GPU operations to be synchronized to ensure accurate timing
tf.config.experimental.set_synchronous_execution(True)

# Set TensorFlow log level to output only errors
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

# Create Model (VGG16)
def create_model():
    inputs = Input(shape=(224, 224, 3))
    # Use VGG16 as the base model without the top layers
    base_model = tf.keras.applications.VGG16(input_shape=(224, 224, 3), include_top=False, weights='imagenet', input_tensor=inputs)
    
    x = base_model(inputs)
    x = GlobalAveragePooling2D()(x)  # Global average pooling
    outputs = Dense(100, activation='softmax')(x)  # Assume 100-class classification
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Get Image Files 
def get_image_files(dataset_dir):
    image_files = []
    for root, dirs, files in os.walk(dataset_dir):
        for file in files:
            if file.endswith(('.png', '.jpg', '.jpeg')):
                image_files.append(os.path.join(root, file))
    return image_files

# Process Image 
def load_and_preprocess_image(image_path):
    img = Image.open(image_path)
    img = img.resize((224, 224))
    img_array = np.array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0
    return img_array

# GPU Synchronization and Cache Clearing 
def gpu_synchronize_and_clear():
    tf.config.experimental.set_synchronous_execution(True)  # Force GPU synchronization
    tf.keras.backend.clear_session()  # Clear GPU cache

# Preload All Images to Memory 
def preload_all_images_to_memory(image_files):
    """Preload all images into memory to avoid frequent disk access during inference."""
    preloaded_images = {}
    for image_file in image_files:
        img_array = load_and_preprocess_image(image_file)
        preloaded_images[image_file] = img_array
    return preloaded_images

# Inference Function 
def inference_with_separate_timing(model, images, num_images):
    gpu_synchronize_and_clear()  # Clear cache and synchronize GPU

    # Perform inference and time it
    t1 = time.time()
    model.predict(images)
    gpu_synchronize_and_clear()  # Synchronize GPU again

    t2 = time.time()
    inference_time = t2 - t1  # Record inference time
    return inference_time

# Single Image Inference 
def single_image_inference(model, image_files):
    random_image_file = random.choice(image_files)
    
    # Record image loading and preprocessing time
    image_data = load_and_preprocess_image(random_image_file)
    
    # Perform inference and record inference time
    inference_time = inference_with_separate_timing(model, image_data, 1)
    return inference_time, random_image_file

# Batch Image Inference 
def batch_image_inference_and_async_loading(model, preloaded_images, image_files, batch_size):
    t1 = time.time()
    for img in random.sample(image_files, batch_size):
        img = preloaded_images[img]
        model.predict(img)
    t2 = time.time()
    inference_time = t2 - t1
    average_time_per_image = inference_time / batch_size
    return inference_time, average_time_per_image

# Main Program Entry 
if __name__ == "__main__":
    dataset_dir = "../data"  # Set dataset directory
    model = create_model()

    # Model warm-up
    model(np.zeros((1, 224, 224, 3)))

    # Get image files
    image_files = get_image_files(dataset_dir)

    # Preload all images into memory
    preloaded_images = preload_all_images_to_memory(image_files)

    # Calculate inference time for 1 image, 500 images, and 1000 images
    for batch_size in [1, 500, 1000]:
        if batch_size == 1:
            inference_time, image_used = single_image_inference(model, image_files)
            print(f"Single image inference time: {inference_time * 1000:.4f} ms")
        else:
            inference_time, avg_time_per_image = batch_image_inference_and_async_loading(model, preloaded_images, image_files, batch_size)
            print(f"Batch of {batch_size} images inference time: {inference_time * 1000:.4f} ms")
            print(f"Average time per image in batch of {batch_size}: {avg_time_per_image * 1000:.4f} ms")
